In [4]:
import os
import pandas as pd
import numpy as np
import sys

# additional paths
feature_path = os.path.join(os.getcwd(), '..', 'src', 'features')
visualize_path = os.path.join(os.getcwd(), '..', 'src', 'visualization')
sys.path.append(feature_path)
sys.path.append(visualize_path)


import build_features # type: ignore
import visualize # type: ignore
from sklearn.feature_selection import VarianceThreshold

In [5]:
data_path = os.path.join('..', 'data', 'interim', '01_H_Contortus_processed.pkl')

df = pd.read_pickle(data_path)
df.head()

,SMILES,ACTIVITY,ACTIVITY_SCORE,ACTIVATION_AT_6.8uM
PUBCHEM_RESULT_TAG,,,,
1,CC1=CC(=NC(=N1)NC2=NC(=CS2)C3=CC=CC=C3)C,1,100.0,36.85
2,CC1=C2C(=CC=C1)C(=C3C(=CC(=NC3=N2)C)C)N,1,84.0,31.28
3,CN1C2=CC=CC=C2N(C1=N)CC(=O)C3=CC=C(C=C3)Br.Br,1,77.0,28.61
4,C1=CC=C2C(=C1)C(=C3C=CC=CC3=N2)NCCCO,1,77.0,28.51
5,CC1=CC2=C(C=C1)N=C(N2)C3=C(C4=C(S3)N=C(C=C4)C)N,1,76.0,28.1


In [6]:
smiles = df['SMILES'].to_list()
morgan_df = build_features.morgan_descriptors(smiles)

In [7]:
morgan_df.head()

,morgan_0,morgan_1,morgan_2,morgan_3,morgan_4,morgan_5,morgan_6,morgan_7,morgan_8,morgan_9,...,morgan_2038,morgan_2039,morgan_2040,morgan_2041,morgan_2042,morgan_2043,morgan_2044,morgan_2045,morgan_2046,morgan_2047
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier

pipe = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('low variance', VarianceThreshold()),
        ('estimator', GradientBoostingClassifier())
    ]
)

clf = RandomForestClassifier(random_state=42)

scores = cross_val_score(pipe, morgan_df, df.ACTIVITY, cv=5, scoring='f1_macro')

In [11]:
scores

array([0.58305744, 0.58542279, 0.6213437 , 0.68033148, 0.66647222])